## About
- This notebook is the exploratory background of a proyect of transaction prediction in digital channels. The objective was to predict the future activity in two channels.
- Fb Prophet is used in all the notebook.
- The databases for training where subseted in different dates and different regresors. The performance for eaach training subset was recorded and a model winner was selected from 3 models, for each channel.
- Functions to iterate trough the 6 models where built for prediction, cross validation and hyperparameter tunning. 
- The CV functions outputs performance metrics that are later fetch by another notebook to get the best model and plot the behavior trough time.

## Imports

In [1]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric, add_changepoints_to_plot
%config InlineBackend.figure_format = 'retina'

#basics
import pandas as pd
import warnings
import itertools
import numpy as np
from datetime import datetime, timedelta
import calendar
import pickle
import glob
import operator

#graphs
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
sys.path.append('../..')

#external aux module
import data_prepro

#set options
pd.set_option('display.float_format', '{:.2f}'.format)
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 400)

#metrics
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

pd.options.mode.chained_assignment = None  # default='warn'

## Data import and prep

In [ ]:
data_prep = pd.read_csv(ruta + 'data_prep.csv') 

In [7]:
# if prediciton is longtherm, prediccion_corta=False
# if prediction is +12 days, prediccion_corta=True
data_modelmb, data_modelab = data_prepro.preprocesamiento(data_prep, prediccion_corta = False)

In [8]:
#correction on variable with some missing values from datalake
data_modelmb['sin_bancos_prox'] = data_modelmb['sin_bancos_prox'].fillna(method='ffill')
data_modelab['sin_bancos_prox'] = data_modelab['sin_bancos_prox'].fillna(method='ffill')

In [9]:
#subsets for channel AB
AB2_data = data_modelab[data_modelab.ds> "2020-03-15"]
AB1_data = data_modelab[data_modelab.ds> "2020-01-05"]
AB3_data = data_modelab[data_modelab.ds> "2018-01-27"]

#subsets for channel MB
MB3_data = data_modelmb[data_modelmb.ds> "2020-03-15"]
MB2_data = data_modelmb[data_modelmb.ds> "2020-01-05"]
MB1_data = data_modelmb[data_modelmb.ds> "2018-05-05"]

### Loading of regresors and hyperparameters from previously trained models
- The following lines, get lists of regresors (.txt files) and hyperparameters (pickle files)

- The commented lines allow to save new regresors or hyperparameters to the same import path.


### Short model - load regs and hypers

In [45]:
# # GUARDAR LOS REGRESORES EN MEMORIA
# for key in regs_dict.keys():
#     #GUARDAR REGRESORES EN UN .TXT
#     with open("hypers_modelo_10dias/regresores/" + key + "_regs.txt", "wb") as fp:   #Pickling
#         pickle.dump(regs_dict[key], fp)

# # GUARDAR LOS HYPERPARAMETROS EN MEMORIA
# for key in hyper_dict.keys():
#     #GUARDAR REGRESORES EN UN .TXT
#     with open("hypers_modelo_10dias/hiperparametros/" + key + "_hyper.txt", "wb") as fp:   #Pickling
#         pickle.dump(hyper_dict[key], fp)
        
    
    
#CARGAR REGRESORES EN MEMORIA
regs_dict =   { 'ab1' :  'to_load' , 'ab2' : 'to_load', 'ab3' : 'to_load' ,'mb1' : 'to_load', 'mb2' : 'to_load', 'mb3' : 'to_load' }
for key in regs_dict.keys():
    with open("hypers_modelo_10dias/regresores/" + key + "_regs.txt", "rb") as fp:   # Unpickling
        regs_dict[key] = pickle.load(fp)
        
#CARGAR HYPERPARAMETROS EN MEMORIA
hyper_dict = {'AB_1': 'to_load','AB_2': 'to_load','AB_3': 'to_load','MB_1': 'to_load','MB_2': 'to_load','MB_3': 'to_load' }
for key in hyper_dict.keys():
    with open("hypers_modelo_10dias/hiperparametros/" + key + "_hyper.txt", "rb") as fp:   # Unpickling
        hyper_dict[key] = pickle.load(fp)      


In [50]:
#SACAMOS REGRESORES DE MODELO CORTO - REFERENCIA "OUT2 sprint 5"

# def remove_regs_from_list(to_remove, base_reg_list):
#     update_list = base_reg_list
    
#     for reg in to_remove:
#         if reg in update_list: update_list.remove(reg)

#     base_reg_list = update_list

# remove_regs_from_list(MB1, regs_dict['ab1'])
# remove_regs_from_list(MB2, regs_dict['ab2'])
# remove_regs_from_list(MB3, regs_dict['ab3'])
# remove_regs_from_list(AB1, regs_dict['mb1'])
# remove_regs_from_list(AB2, regs_dict['mb2'])
# remove_regs_from_list(AB3, regs_dict['mb3'])

In [46]:

data_hypers_AB_1b = {'data':AB1_data, 'hypers':hyper_dict['AB_1'], 'name_mod': 'AB_1b', 'regresores':regs_dict['ab1']  } 
data_hypers_AB_2 = {'data':AB2_data, 'hypers':hyper_dict['AB_2'], 'name_mod': 'AB_2', 'regresores':regs_dict['ab2'] }
data_hypers_AB_3 = {'data':AB3_data, 'hypers':hyper_dict['AB_3'], 'name_mod': 'AB_3' , 'regresores':regs_dict['ab3'] }
data_hypers_MB_1b = {'data':MB1_data, 'hypers':hyper_dict['MB_1'], 'name_mod':  'MB_1b', 'regresores':regs_dict['mb1'] }
data_hypers_MB_2 = {'data':MB2_data, 'hypers':hyper_dict['MB_2'], 'name_mod':  'MB_2', 'regresores': regs_dict['mb2'] }
data_hypers_MB_3 = {'data':MB3_data, 'hypers':hyper_dict['MB_3'] , 'name_mod': 'MB_3', 'regresores': regs_dict['mb3'] }


#lista iterable de diccionarios
to_predict_dict = [
data_hypers_AB_1b,
data_hypers_AB_2,
data_hypers_AB_3,
data_hypers_MB_1b,
data_hypers_MB_2,
data_hypers_MB_3,
data_hypers_MB_2,
data_hypers_MB_3
]



### Long model -  - load regs and hypers

In [10]:
# #GUARDAR LOS REGRESORES EN MEMORIA
# with open("hypers_modelolargo/regresores/regresoresAB.txt", "wb") as fp:   #Pickling
#     pickle.dump(regs_long_ab, fp)
# with open("hypers_modelolargo/regresores/regresoresMB.txt", "wb") as fp:   #Pickling
#     pickle.dump(regs_longs_mb, fp)

# # GUARDAR LOS HYPERPARAMETROS EN MEMORIA
# for key in hyper_dict_long.keys():
#     #GUARDAR REGRESORES EN UN .TXT
#     with open("hypers_modelolargo/hiperparametros/" + key + "_hyper.txt", "wb") as fp:   #Pickling
#         pickle.dump(hyper_dict_long[key], fp)


        
#CARGAR HIPERPARAMETROS EN MEMORIA
hyper_dict_long = {'AB1': 'to_load','AB3' : 'to_load','MB1': 'to_load','MB2': 'to_load','MB3': 'to_load' }
for key in hyper_dict_long.keys():
    with open("hypers_modelolargo/hiperparametros/" + key + "_hyper.txt", "rb") as fp:   # Unpickling
        hyper_dict_long[key] = pickle.load(fp)      


# CARGAR REGRESORES EN MEMORIA
# #cargamos los regresores en memoria
        
with open("hypers_modelolargo/regresores/regresoresAB.txt", "rb") as fp:   # Unpickling
    regs_long_ab = pickle.load(fp)

with open("hypers_modelolargo/regresores/regresoresMB.txt", "rb") as fp:   # Unpickling
    regs_longs_mb = pickle.load(fp)

In [11]:
#generamos los diccionarios con todos los modelos que fueron quedando y se re validan cada vez.


#ruta2 = 'hypers_modelolargo\\'
ruta2 = 'hypers_modelolargo/hiperparametros/'

#diccionarios con datos, hiperparámetros y nombre de los modelos
data_AB1  = {'data':AB1_data, 'hypers':hyper_dict_long['AB1'], 'name_mod': 'AB_1', 'regresores':regs_long_ab} 
data_AB3 = {'data':AB3_data, 'hypers':hyper_dict_long['AB3'] , 'name_mod': 'AB_3' , 'regresores':regs_long_ab}
data_MB1  = {'data':MB1_data, 'hypers':hyper_dict_long['MB1'] , 'name_mod':  'MB_1', 'regresores':regs_longs_mb  }
data_MB2 = {'data':MB2_data, 'hypers':hyper_dict_long['MB2'], 'name_mod':  'MB_2', 'regresores':regs_longs_mb }
data_MB3 = {'data':MB3_data, 'hypers':hyper_dict_long['MB3'] , 'name_mod': 'MB_3', 'regresores':regs_longs_mb}

#lista iterable de diccionarios para pasar al modelo
to_predict_dict_long = [
data_AB1,
data_MB3
]

In [20]:
pd.DataFrame(data_MB3)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

## Prediction

### Short Model - Prediction

In [47]:
# Function that runs a prediction for the current model in the iterated list.

def massive_prediction(predict_days, data, hyperparams, name,regs):

    #instanciamos el modelo
    m = Prophet(**hyperparams)
    
    #le pasamos los regresores al modelo instanciado con el metodo add regressor
    for item in regs:
        m.add_regressor(item, mode='additive')
      
    datos_modelo = data
    df_train = datos_modelo[:-predict_days] #subset train
    df_test = datos_modelo[-predict_days:] #subset prediction

    m.fit(df_train)
    forecast = m.predict(df_test) #matriz de prediccion
    forecast['sem_dia'] = forecast['ds'].dt.strftime('%a') #weekends drops
    
    forecast = forecast[(forecast.sem_dia != 'Sat') & (forecast.sem_dia != 'Sun')]
    
    # file name information
    prediccion_hasta = df_test.ds.iloc[-1].strftime("%d-%b-%Y")
    
   #model name
    forecast['model'] = name
    
    forecast.to_csv( 'predicciones/' + name +' '+ prediccion_hasta + '.csv')
    print('csv saved') #guardamos tabla predict con info completa
    
    forecast['yhat'] = forecast['yhat'].astype(int)
    forecast['canal'] = name[0:2]
    
    #returns prediction report
    return forecast[['ds', 'yhat', 'canal']] ,prediccion_hasta
    

In [48]:
#iterates the massive_prediction for the model list. 

#generates 2 outputs:
#model prediction with regresor input for each prediction
#clean csv with model prediction for client

to_predict_dict = [data_hypers_AB_1b,data_hypers_MB_2]

summary_list = pd.DataFrame()
for i in to_predict_dict:
    print(i['name_mod'])
    model_summary, prediccion_hasta  = massive_prediction(predict_days=i['data']['dias_predict'].iloc[0],
                                       data=i['data'], hyperparams=i['hypers'],
                                       name=i['name_mod'],
                                       regs=i['regresores'])
   
    summary_list = summary_list.append(model_summary, ignore_index=True)
summary_list.rename(columns = {'ds': 'fecha', 'yhat' : 'prediccion_trx'}, inplace=True)
summary_list.to_csv( 'predicciones/prediccion_corta/' + 'prediccion_' +prediccion_hasta + '.csv')

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


AB_1b


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


csv saved
MB_2
csv saved


### Long Model - Prediction

In [69]:
# same as before but for long model.

def massive_prediction(predict_days, data, hyperparams, name,regs):


    m = Prophet(**hyperparams) 

   
    for item in regs:
        m.add_regressor(item, mode='additive')
      
    datos_modelo = data
    df_train = datos_modelo[:-predict_days]
    df_test = datos_modelo[-predict_days:] 

    m.fit(df_train) 
    forecast = m.predict(df_test) 
    
    forecast['sem_dia'] = forecast['ds'].dt.strftime('%a') 
    forecast = forecast[(forecast.sem_dia != 'Sat') & (forecast.sem_dia != 'Sun')]
    
    prediccion_hasta = df_test.ds.iloc[-1].strftime("%d-%b-%Y")
  
    forecast.to_csv( 'predicciones/prediccion_larga/' + name +' '+ prediccion_hasta + '.csv')
    print('csv saved') #guardamos tabla predict con info completa
    
    forecast['yhat'] = forecast['yhat'].astype(int)
    
    forecast['canal'] = name[0:2]
    
    return forecast[['ds', 'yhat', 'canal']] ,prediccion_hasta

In [70]:
#same as before but for long model.

to_predict_dict = [data_AB1,data_MB3] 

summary_list = pd.DataFrame() 
for i in to_predict_dict:
    print(i['name_mod'])
    model_summary, prediccion_hasta  = massive_prediction(predict_days=MB1_data.isna().sum().y,
                                       data=i['data'], hyperparams=i['hypers'],
                                       name=i['name_mod'],
                                       regs=i['regresores'])
   
    summary_list = summary_list.append(model_summary, ignore_index=True) 
summary_list.rename(columns = {'ds': 'fecha', 'yhat' : 'prediccion_trx'}, inplace=True) 
summary_list.to_csv( 'predicciones\\prediccion_larga\\entregable\\' + 'prediccion_' +prediccion_hasta + '.csv') 

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


AB_1


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


csv saved
MB_3
csv saved


## Cross-validation testing and hyperparameter tunning

### PREDICT CV - TESTING

In [41]:
#function that iterates cross validation over the selected models.

id_mod = 1
summary_list = pd.DataFrame()
for i in to_predict_dict_long:
        id_mod = id_mod + 1
        
        model_summary = massive_prediction_cv(
            initial_forecast_date=i['data'].ds.iloc[-i['data'].y.isna().sum()] - timedelta(100)
,
            data=i['data'],
            hyperparams=i['hypers'],
            name=i['name_mod'], 
            regs= i['regresores'])  
        summary_list = summary_list.append(model_summary, ignore_index=True)
        
summary_list.to_csv(
    '../../documents/logs/produccion_test/cv_test/summary/summary_list_cv.csv')
print('finish')

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 15 forecasts with cutoffs between 2020-06-08 00:00:00 and 2020-10-26 00:00:00


148 days


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.



AB_1


INFO:fbprophet:Making 15 forecasts with cutoffs between 2020-06-08 00:00:00 and 2020-10-26 00:00:00


78 days



MB_3
finish


In [40]:
def massive_prediction_cv(initial_forecast_date, data, hyperparams, name, regs):


    m = Prophet() 
    

    
    for item in regs:
        m.add_regressor(item, mode='additive')
    
    m.fit(data)
    
    initial = str(abs(data[data.ds> initial_forecast_date].shape[0] - data.shape[0])) + ' days'
    print(initial) 
    
    df_cv = cross_validation(m, initial=initial, period='10 days', horizon = '10 days')
    
    
    df_cv['sem_dia'] = df_cv['ds'].dt.strftime('%a')
    df_cv = df_cv[(df_cv.sem_dia != 'Sat') & (df_cv.sem_dia != 'Sun')]
    df_cv = df_cv[df_cv['y'] != 0]
    
 
    df_p = performance_metrics(df_cv)
    rmse = df_p.rmse.mean()
    mae =  df_p.mae.mean()
    mape = df_p.mape.mean()
    model_sum = {'rmse':rmse,
                'mae':mae,
                'mape':mape,
                'name': name,
                }
    print(name)
    
    df_cv['model'] = name
    df_cv['sem_dia'] = df_cv['ds'].dt.strftime('%a')
    df_cv.to_csv('../../documents/logs/produccion_test/cv_test/' + name +'.csv')

    return model_sum #devuelve el summary en formato de diccionario

### HYPER-PARAMETER TUNNING

In [7]:
for model in to_predict_dict:
    
    print('tunning params on model'+ model['name_mod']) 
    
    data = model['data']
    regresors_additive = model['regresores']

    param_grid = {  
        'changepoint_prior_scale': [0.05, 0.001, 0.01, 0.1, 0.33, 0.5], #default 0.05
        'seasonality_prior_scale': [10.0, 0.01, 0.1, 1.0], #default 10
       # 'n_changepoints': [1,2,5,10,15], 
        'changepoint_range': [0.8,0.5, 0.9], #hasta que punto busca changepoints 
        #'yearly_seasonality': ["auto"],
        #'weekly_seasonality': ["auto"],
        'daily_seasonality': [False],
        #'holidays': [holidays],
        'seasonality_mode': ["additive", "multiplicative"],
        #'holidays_prior_scale': [10, 50, 100],
        #'mcmc_samples': [0],
        'interval_width': [0.80], #defualt 0.8
        #'uncertainty_samples': [0] 
      }


    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params)


        for item in regresors_additive:
            m.add_regressor(item, mode='additive')
        #for item in regresors_multiplicative:
        #    m.add_regressor(item, mode='multiplicative')

        m.fit(data)  # Fit model with given params

        initial_forecast_date = '2020-08-01'
        initial = str(abs(data[data.ds> initial_forecast_date].shape[0] - data.shape[0])) + ' days'


        df_cv = cross_validation(m,initial=initial, period='10 days', horizon = '10 days', parallel="processes")

        df_cv['sem_dia'] = df_cv['ds'].dt.strftime('%a')
        df_cv = df_cv[(df_cv.sem_dia != 'Sat') & (df_cv.sem_dia != 'Sun')]

        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    #Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses
    print(tuning_results)
    best_params = all_params[np.argmin(rmses)]
    print(best_params)
    
    #actualizamos los hiperparámetros
    print('saving new hypers')
    model['hypers'] = best_params
    

tunning params on modelAB_1b


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a7fc70>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a7adf0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77399f7b20>
INFO:fbprophet:Disabling yearly seasonality. R

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739988fa0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a39580>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a043a0>
INFO:fbprophet:Disabling yearly seasonality. R

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a04cd0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a21cd0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77399fd880>
INFO:fbprophet:Disabling yearly seasonality. R

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77399d8d60>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f773b42d610>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77399ee3d0>
INFO:fbprophet:Disabling yearly seasonality. R

...

INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739988c70>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77355f4970>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a57b50>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs

INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a39520>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a57b50>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739408400>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs

INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77355e77c0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77399f4f70>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a149d0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs

INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f7739a181f0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f77355f4970>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs between 2020-08-07 00:00:00 and 2020-10-26 00:00:00
INFO:fbprophet:Applying in parallel with <concurrent.futures.process.ProcessPoolExecutor object at 0x7f773998f0d0>
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Making 9 forecasts with cutoffs

     changepoint_prior_scale  seasonality_prior_scale  changepoint_range  \
0                       0.05                    10.00               0.80   
1                       0.05                    10.00               0.80   
2                       0.05                    10.00               0.50   
3                       0.05                    10.00               0.50   
4                       0.05                    10.00               0.90   
5                       0.05                    10.00               0.90   
6                       0.05                     0.01               0.80   
7                       0.05                     0.01               0.80   
8                       0.05                     0.01               0.50   
9                       0.05                     0.01               0.50   
10                      0.05                     0.01               0.90   
11                      0.05                     0.01               0.90   
12          

Modelo corto - re asignamos los hypers al diccionario y los guardamos en pickles en la carpeta correspondiente

In [10]:
#DESPUES DE CORRER HYPER TUNNING, ASIGNO
hyper_dict = {'AB_1': to_predict_dict[0]['hypers'], 'AB_2': to_predict_dict[1]['hypers'],'AB_3': to_predict_dict[2]['hypers'],'MB_1': to_predict_dict[3]['hypers'],'MB_2': to_predict_dict[4]['hypers'],'MB_3': to_predict_dict[5]['hypers']}
# GUARDAR LOS HYPERPARAMETROS EN MEMORIA
for key in hyper_dict.keys():
    #GUARDAR REGRESORES EN UN .TXT
    with open("hypers_modelo_10dias/hiperparametros/" + key + "_hyper.txt", "wb") as fp:   #Pickling
        pickle.dump(hyper_dict[key], fp)

{'changepoint_prior_scale': 0.1,
 'seasonality_prior_scale': 0.01,
 'changepoint_range': 0.5,
 'daily_seasonality': False,
 'seasonality_mode': 'multiplicative',
 'interval_width': 0.8}

In [14]:
hyper_dict['AB_1']

{'changepoint_prior_scale': 0.05,
 'seasonality_prior_scale': 10.0,
 'changepoint_range': 0.8,
 'daily_seasonality': False,
 'seasonality_mode': 'additive',
 'interval_width': 0.8}

In [15]:
hyper_dict['AB_3']

{'changepoint_prior_scale': 0.001,
 'seasonality_prior_scale': 1.0,
 'changepoint_range': 0.5,
 'daily_seasonality': False,
 'seasonality_mode': 'additive',
 'interval_width': 0.8}

### PROPHET - ITERACION REGRESORES

In [8]:
#auxiliary funciton - builds a list with all the possible combination of different regresors. Then iterate over all possible combinations and get the best set of regresors
#according to model performance in cross validation.

def regresores_cobinatoria(regs_list=None):
    if regs_list==None:
        return None
    else:
        l = []
        from itertools import chain, combinations
        a = regs_list
        def all_subsets(ss):
            return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

        for subset in all_subsets(a):
            #print(list(subset))
            l.append(list(subset))
        return l